RDFLib aims to be a pythonic RDF API, a Graph is a python collection of RDF 
Subject, Predicate, Object Triples.

In [2]:
import rdflib
g = rdflib.Graph()

result = g.parse('epigraphy.ttl', format="turtle")

In [3]:
#Print the length of your ttl 
print(len(g))    

23


In [4]:
for s, p, o in g:
    print(s, p, o)

http://aph/vocab/name http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Property
http://aph/ids/0000490 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://aph/vocab/person
http://aph/vocab/person http://www.w3.org/2000/01/rdf-schema#label Person
http://aph/ids/0000956 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://aph/vocab/person
http://aph/ids/a000013 http://aph/vocab/name anonymous
http://aph/ids/0000441 http://aph/vocab/name Aristokles Molossos
http://aph/ids/0000441 http://aph/vocab/hasfather http://aph/ids/0000440
http://aph/vocab/person http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://aph/ids/a000013 http://aph/vocab/hasfather http://aph/ids/0000440
http://aph/vocab/name http://www.w3.org/2000/01/rdf-schema#label Name
http://aph/ids/0000440 http://aph/vocab/hasfather http://aph/ids/0000490
http://aph/ids/0001383 http://aph/vocab/hasfather http://aph/ids/0000490
http://aph/ids/0000956 

In [5]:
#try this to see their type
#for stmt in g:
#    print(stmt)

The components of the triples are URIs (resources) or Literals (values), URIs are grouped together by namespace, common namespaces are included in RDFLib:

In [6]:
semweb=rdflib.URIRef("epigraphy.ttl")
type=g.value(semweb, rdflib.RDFS.label)

Print each person whose father's name is preserved

In [7]:
qres = g.query(
    """SELECT ?s
    WHERE 
    {
    ?s aphvocab:hasfather ?o .
    } 
    """)

for row in qres:
    print("The name of %s's father is preserved" % row )

The name of http://aph/ids/0001383's father is preserved
The name of http://aph/ids/0000440's father is preserved
The name of http://aph/ids/0000441's father is preserved
The name of http://aph/ids/0000956's father is preserved
The name of http://aph/ids/a000013's father is preserved


The name of each person in prosopography is

In [8]:
qres = g.query(
    """SELECT ?s ?o
    WHERE 
    {
    ?s aphvocab:name ?o .
    } 
    """)

for row in qres:
    print("The name of %s is %s" % (row.s, row.o))

The name of http://aph/ids/a000013 is anonymous
The name of http://aph/ids/0000441 is Aristokles Molossos
The name of http://aph/ids/0000440 is Aristokles Molossos
The name of http://aph/ids/0000490 is Artemidoros
The name of http://aph/ids/0000956 is Ermas Molossos
The name of http://aph/ids/0001383 is Kaikos Papias


Print the id of each person whose father's name is preserved along with his father's id

In [9]:
qres = g.query(
    """SELECT ?s ?o
    WHERE 
    {
    ?s aphvocab:hasfather ?o .
    } 
    """)

for row in qres:
    print("The name of %s is %s" % (row.s, row.o))

The name of http://aph/ids/0001383 is http://aph/ids/0000490
The name of http://aph/ids/0000440 is http://aph/ids/0000490
The name of http://aph/ids/0000441 is http://aph/ids/0000440
The name of http://aph/ids/0000956 is http://aph/ids/0000440
The name of http://aph/ids/a000013 is http://aph/ids/0000440


Print all the persons with their id and the id of their fathers.

In [10]:
qres = g.query(
    """SELECT ?person ?name ?father
    WHERE 
    {
    ?person aphvocab:name ?name ;
            aphvocab:hasfather ?father .    
    }
    """)

for row in qres:
    print("%s (%s) is son of %s" % (row.name, row.person.replace("http://aph/ids/",""), row.father.replace("http://aph/ids/","")))

Kaikos Papias (0001383) is son of 0000490
Aristokles Molossos (0000440) is son of 0000490
Aristokles Molossos (0000441) is son of 0000440
Ermas Molossos (0000956) is son of 0000440
anonymous (a000013) is son of 0000440


Find the name of the father

In [11]:
qres = g.query(
    """SELECT ?person ?name
    WHERE 
    {
    ?person (aphvocab:hasfather/aphvocab:name) ?name
    }        
    """)

for row in qres:
    print("%s is son of %s" % (row.person, row.name))

http://aph/ids/0001383 is son of Artemidoros
http://aph/ids/0000440 is son of Artemidoros
http://aph/ids/0000441 is son of Aristokles Molossos
http://aph/ids/0000956 is son of Aristokles Molossos
http://aph/ids/a000013 is son of Aristokles Molossos


Person (id) is the son of Person (id)

In [84]:
qres = g.query(
    """SELECT ?person ?name ?father ?fathersname
    WHERE 
    {
    ?person aphvocab:name ?name ;
            aphvocab:hasfather ?father ;
            aphvocab:hasfather [ aphvocab:name ?fathersname ] .
    }
   
   ORDER BY ?person
   """)

for row in qres:
    print("%s (%s) is son of %s (%s)" % (row.name, row.person.replace("http://aph/ids/",""), row.fathersname, row.father.replace("http://aph/ids/","")))

Aristokles Molossos (0000440) is son of Artemidoros (0000490)
Aristokles Molossos (0000441) is son of Aristokles Molossos (0000440)
Ermas Molossos (0000956) is son of Aristokles Molossos (0000440)
Kaikos Papias (0001383) is son of Artemidoros (0000490)
anonymous (a000013) is son of Aristokles Molossos (0000440)


Person (id) is the son of Person (id) and the grandson of Person (id) (1st try)

In [83]:
qres = g.query(
    """SELECT ?person ?name ?father ?fathersname ?grandfather ?grandfathername 
    WHERE 
    {
    ?person aphvocab:name ?name ;
            aphvocab:hasfather ?father ;
            aphvocab:hasfather [ aphvocab:name ?fathersname ] ;
            aphvocab:hasfather [ aphvocab:hasfather ?grandfather ] ;
            aphvocab:hasfather [ aphvocab:name ?grandfathername ] .

    }
    ORDER BY ?person
    """)

for row in qres:
    print("%s (%s) is son of %s (%s), son of %s (%s)" % (row.name, row.person.replace("http://aph/ids/",""), row.fathersname, row.father.replace("http://aph/ids/",""), row.grandfathername, row.grandfather.replace("http://aph/ids/","")))

Aristokles Molossos (0000441) is son of Aristokles Molossos (0000440), son of Artemidoros (0000490)
Ermas Molossos (0000956) is son of Aristokles Molossos (0000440), son of Artemidoros (0000490)
anonymous (a000013) is son of Aristokles Molossos (0000440), son of Artemidoros (0000490)


Person (id) is the son of Person (id) and the grandson of Person (id) (2st try)

In [386]:
qres = g.query(
    """SELECT ?person ?name ?father ?fathersname ?grandfather ?grandfathername 
    WHERE 
    {
    ?person aphvocab:name ?name .
    OPTIONAL
    { 
    ?person aphvocab:hasfather ?father ;
            aphvocab:hasfather ?father ;
            aphvocab:hasfather [ aphvocab:name ?fathersname ] ;
            aphvocab:hasfather [ aphvocab:hasfather ?grandfather ] ;
            aphvocab:hasfather [ aphvocab:name ?grandfathername ]
    } .
    }
    ORDER BY ?person
    """)

for row in qres:
    print("%s (%s) is son of %s (%s), son of %s (%s)" % (row.name, row.person, row.fathersname, row.father, row.grandfathername, row.grandfather))

Aristokles Molossos (http://aph/ids/0000440) is son of None (None), son of None (None)
Aristokles Molossos (http://aph/ids/0000441) is son of Aristokles Molossos (http://aph/ids/0000440), son of Artemidoros (http://aph/ids/0000490)
Artemidoros (http://aph/ids/0000490) is son of None (None), son of None (None)
Ermas Molossos (http://aph/ids/0000956) is son of Aristokles Molossos (http://aph/ids/0000440), son of Artemidoros (http://aph/ids/0000490)
Kaikos Papias (http://aph/ids/0001383) is son of None (None), son of None (None)
anonymous (http://aph/ids/a000013) is son of Aristokles Molossos (http://aph/ids/0000440), son of Artemidoros (http://aph/ids/0000490)


TEST of VALUES

In [288]:

qres = g.query(
    """SELECT ?predicate ?person
    WHERE 
    {
    }
    VALUES (?predicate ?person)
    {
    (aphvocab:hasfather "http://aph/ids/0000440" )
    }
    """)

for row in qres:
    print(row.predicate.replace("http://aph/vocab/",""), row.person.replace("http://aph/ids/",""))

hasfather 0000440


In [289]:
qres = g.query(
    """SELECT ?person ?name ?father ?fathersname
    WHERE 
    {
    ?person aphvocab:name ?name ;
            aphvocab:hasfather ?father ;
            aphvocab:hasfather [ aphvocab:name ?fathersname ] .
    }
   
   ORDER BY ?person
   """)

for row in qres:
    print("%s (%s) \t is son of %s (%s)" % (row.name, row.person.replace("http://aph/ids/",""), row.fathersname, row.father.replace("http://aph/ids/","")))

Aristokles Molossos (0000440) 	 is son of Artemidoros (0000490)
Aristokles Molossos (0000441) 	 is son of Aristokles Molossos (0000440)
Ermas Molossos (0000956) 	 is son of Aristokles Molossos (0000440)
Kaikos Papias (0001383) 	 is son of Artemidoros (0000490)
anonymous (a000013) 	 is son of Aristokles Molossos (0000440)


In [315]:
qres = g.query(
    """SELECT ?person ?name ?father ?fathersname
    WHERE 
    {
    ?person aphvocab:name ?name ;
            aphvocab:hasfather ?father ;
            aphvocab:hasfather [ aphvocab:name ?fathersname ] .
    }
   
   ORDER BY ?person
   """)

for row in qres:
    print(row.name, row.person.replace("http://aph/ids/",""), row.fathersname, row.father.replace("http://aph/ids/",""))

Aristokles Molossos 0000440 Artemidoros 0000490
Aristokles Molossos 0000441 Aristokles Molossos 0000440
Ermas Molossos 0000956 Aristokles Molossos 0000440
Kaikos Papias 0001383 Artemidoros 0000490
anonymous a000013 Aristokles Molossos 0000440


In [361]:
qres = g.query(
   
    """SELECT ?name ?person ?fathersname ?father 
    WHERE 
    {
    ?person aphvocab:name ?name ;
            aphvocab:hasfather ?father ;
            aphvocab:hasfather [ aphvocab:name ?fathersname ] .
    }
   
   ORDER BY ?person
   """)



template = "{0:30}|{1:30}|{2:30}|{3:30}" #column widths: 8, 10, 15, 7, 10
print(template.format("NAME", "ID", "FATHER'S NAME", "FATHER'S ID")) # header
for rec in qres: 
    print(template.format(*rec))

NAME                          |ID                            |FATHER'S NAME                 |FATHER'S ID                   
Aristokles Molossos           |http://aph/ids/0000440        |Artemidoros                   |http://aph/ids/0000490        
Aristokles Molossos           |http://aph/ids/0000441        |Aristokles Molossos           |http://aph/ids/0000440        
Ermas Molossos                |http://aph/ids/0000956        |Aristokles Molossos           |http://aph/ids/0000440        
Kaikos Papias                 |http://aph/ids/0001383        |Artemidoros                   |http://aph/ids/0000490        
anonymous                     |http://aph/ids/a000013        |Aristokles Molossos           |http://aph/ids/0000440        


In [323]:
#template = "{0:30}|{1:30}|{2:30}|{3:30}|{4:30}|{5:30}" #column widths: 8, 10, 15, 7, 10
#print(template.format("NAME", "ID", "FATHER'S NAME", "FATHER'S ID", "GRANDFATHER", "GRANDFATHER'S ID")) # header
#for rec in qres: 
#      print(template.format(*rec))

In [442]:
qres = g.query(
    """SELECT ?person ?name ?father ?fathersname ?grandfather ?grandfathername 
    WHERE 
    {
    ?person aphvocab:name ?name .
    OPTIONAL
    { 
    ?person aphvocab:hasfather ?father ;
            aphvocab:hasfather ?father ;
            aphvocab:hasfather [ aphvocab:name ?fathersname ] ;
            aphvocab:hasfather [ aphvocab:hasfather ?grandfather ] ;
            aphvocab:hasfather [ aphvocab:name ?grandfathername ]
    } .
    }
    ORDER BY ?person
    """)

template = "{0!s:20}\t{1!s:20}\t{2!s:20}\t{3!s:20}\t{4!s:20}\t{5!s:20}" #column widths: 8, 10, 15, 7, 10
print(template.format("NAME", "ID", "FATHER'S NAME", "FATHER'S ID", "GRANDFATHER", "GRANDFATHER'S ID")) # header
for row in qres:
    if row == None:
        str(None)
    print(template.format(row.name, row.person, row.fathersname, row.father, row.grandfathername, row.grandfather))


#template = "{0:20}|{1:20}|{2:20}|{3:20}|{4:20}|{5:20}" #column widths: 8, 10, 15, 7, 10
#*rec.replace("http://aph/ids/","")
#print(template.format("NAME", "ID", "FATHER'S NAME", "FATHER'S ID", "GRANDFATHER", "GRANDFATHER'S ID")) # header
#for rec in qres: 
#    print(row.name, row.person.replace("http://aph/ids/",""), row.fathersname, row.father.replace("http://aph/ids/",""), row.grandfathername, row.grandfather.replace("http://aph/ids/",""))

NAME                	ID                  	FATHER'S NAME       	FATHER'S ID         	GRANDFATHER         	GRANDFATHER'S ID    
Aristokles Molossos 	http://aph/ids/0000440	None                	None                	None                	None                
Aristokles Molossos 	http://aph/ids/0000441	Aristokles Molossos 	http://aph/ids/0000440	Artemidoros         	http://aph/ids/0000490
Artemidoros         	http://aph/ids/0000490	None                	None                	None                	None                
Ermas Molossos      	http://aph/ids/0000956	Aristokles Molossos 	http://aph/ids/0000440	Artemidoros         	http://aph/ids/0000490
Kaikos Papias       	http://aph/ids/0001383	None                	None                	None                	None                
anonymous           	http://aph/ids/a000013	Aristokles Molossos 	http://aph/ids/0000440	Artemidoros         	http://aph/ids/0000490
